In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

# manage ray's relative imports
import ray
runtime_env = {"working_dir": ".." }
ray.init(runtime_env=runtime_env, dashboard_port=13065, include_dashboard=True)

from ray import tune
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune import JupyterNotebookReporter

# manage beams's relative imports
import sys
sys.path.append('..')

from src.beam import beam_arguments, Experiment, Study
from src.beam import UniversalDataset, UniversalBatchSampler
from src.beam import Algorithm
from src.beam import LinearNet
from torchvision import transforms
import matplotlib.pyplot as plt

from src.beam import DataTensor
from src.beam.utils import is_notebook
from cifar10_example import cifar10_algorithm_generator, Cifar10Network

from ray.tune.suggest.hebo import HEBOSearch

2022-06-20 21:04:11,478	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:13065
2022-06-20 21:04:12,533	INFO packaging.py:388 -- Creating a file package for local directory '..'.
2022-06-20 21:04:12,664	INFO packaging.py:241 -- Pushing file package 'gcs://_ray_pkg_ff238c99431a7da4.zip' (1.07MiB) to Ray cluster...
2022-06-20 21:04:12,688	INFO packaging.py:243 -- Successfully pushed file package 'gcs://_ray_pkg_ff238c99431a7da4.zip'.


In [2]:
path_to_data = '/localdata/elads/data/datasets/cifar10'
root_dir = '/localdata/elads/data/cifar10'

# path_to_data = '/home/shared/data/dataset/cifar10'
# root_dir = '/home/shared/data/results/cifar10'

## Training with a single worker

In [34]:
# here you put all actions which are performed only once before initializing the workers
# for example, setti`ng running arguments and experiment:

args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=1 --half --lr-d=1e-4 --batch-size=512",
                      "--n-epochs=40 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=.00256 --beta1=0.9 --beta2=0.9", 
                      path_to_data=path_to_data, gamma=1., dropout=.0, activation='celu', channels=512,
                      padding=4, gain=.4, turn_point=500, final_point=3000, minimal_gain=.02)

experiment = Experiment(args)

2022-06-20 20:48:26.073 | INFO     | src.beam.experiment:__init__:186 - beam project: cifar10
2022-06-20 20:48:26.074 | INFO     | src.beam.experiment:__init__:187 - Experiment Hyperparameters
2022-06-20 20:48:26.075 | INFO     | src.beam.experiment:__init__:192 - project_name: cifar10
2022-06-20 20:48:26.076 | INFO     | src.beam.experiment:__init__:192 - identifier: debug
2022-06-20 20:48:26.077 | INFO     | src.beam.experiment:__init__:192 - algorithm: CIFAR10Algorithm
2022-06-20 20:48:26.078 | INFO     | src.beam.experiment:__init__:192 - mp_port: None
2022-06-20 20:48:26.079 | INFO     | src.beam.experiment:__init__:192 - root_dir: /localdata/elads/data/cifar10
2022-06-20 20:48:26.079 | INFO     | src.beam.experiment:__init__:192 - reload: False
2022-06-20 20:48:26.080 | INFO     | src.beam.experiment:__init__:192 - resume: -1
2022-06-20 20:48:26.081 | INFO     | src.beam.experiment:__init__:192 - override: False
2022-06-20 20:48:26.082 | INFO     | src.beam.experiment:__init__:19

## Train with single or multiple workers

In [35]:
alg = experiment(cifar10_algorithm_generator)

2022-06-20 20:48:28.726 | INFO     | src.beam.experiment:run:554 - Single worker mode
2022-06-20 20:48:28.728 | INFO     | src.beam.experiment:run_worker:48 - Worker: 1/1 is running...


train:   0%|          | 0/73 [00:00<?, ?it/s]

validation:   0%|          | 0/22 [00:00<?, ?it/s]


2022-06-20 20:48:34.585 | INFO     | src.beam.experiment:save_model_results:400 - Finished epoch 1/40:
2022-06-20 20:48:34.587 | INFO     | src.beam.experiment:log_data:446 - train:
2022-06-20 20:48:34.588 | INFO     | src.beam.experiment:log_data:450 - loss 1111.49 	|
2022-06-20 20:48:34.589 | INFO     | src.beam.experiment:log_data:450 - acc 0.287377 	|
2022-06-20 20:48:34.590 | INFO     | src.beam.experiment:log_data:450 - lr 5.84e-06 	|
2022-06-20 20:48:34.590 | INFO     | src.beam.experiment:log_data:446 - validation:
2022-06-20 20:48:34.591 | INFO     | src.beam.experiment:log_data:450 - loss 1039.64 	|
2022-06-20 20:48:34.592 | INFO     | src.beam.experiment:log_data:450 - acc 0.362837 	|


RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 1200384 vs 1200272

In [29]:
np.mean(alg.evaluate('test')['scalar']['acc'])

test:   0%|          | 0/20 [00:00<?, ?it/s]

0.9014935672283173

## Hyperparameter search with native optuna

In [ ]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=1 --half --lr-d=1e-4 --batch-size=512",
                      "--n-epochs=40 --epoch-length-train=50000 --epoch-length-eval=10000 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=.00256 --beta1=0.9 --beta2=0.9", 
                      path_to_data=path_to_data, gamma=1., dropout=.0, activation='celu', channels=512,
                      scale_down=.7, scale_up=1.4, ratio_down=.7, ratio_up=1.4)

study = Study(cifar10_algorithm_generator, args)

In [14]:
def suggest(trial):
    lr = trial.suggest_loguniform("lr", 1e-3, 2e-2)
    print('My suggestion')
    print(lr)
    return {'lr_dense': lr}
    

In [15]:
study.optuna(suggest, direction='maximize', n_jobs=1, n_trials=10)

[I 2022-06-20 08:46:21,221] A new study created in memory with name: cifar10/CIFAR10Algorithm/debug_hp_optimization_20220620_084618


My suggestion
0.009781018192999246
2022-06-20 08:46:21.224 | INFO     | src.beam.experiment:runner_optuna:114 - Next Hyperparameter suggestion:
2022-06-20 08:46:21.225 | INFO     | src.beam.experiment:runner_optuna:116 - lr_dense: 0.009781018192999246
2022-06-20 08:46:21.229 | INFO     | src.beam.experiment:__init__:262 - Creating new experiment
2022-06-20 08:46:21.229 | INFO     | src.beam.experiment:__init__:277 - Experiment directory is: /home/shared/data/results/cifar10/cifar10/CIFAR10Algorithm/debug_hp_optimization_20220620_084618/0000_20220620_084621
2022-06-20 08:46:21.239 | INFO     | src.beam.experiment:run:554 - Single worker mode
2022-06-20 08:46:21.240 | INFO     | src.beam.experiment:run_worker:48 - Worker: 1/1 is running...
0.08
0.16
0.24
0.32
0.4
0.38857142857142857
0.37714285714285717
0.3657142857142857
0.3542857142857143
0.34285714285714286
0.33142857142857146
0.32
0.3085714285714286


KeyboardInterrupt: 

## Hyperparameter search with ray-tune and optuna

In [3]:
args = beam_arguments(f"--project-name=cifar10 --root-dir={root_dir} --algorithm=CIFAR10Algorithm --device=0 --half --lr-d=1e-3 --batch-size=512",
                      "--n-epochs=40 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
                      "--weight-decay=.00256 --beta1=0.9 --beta2=0.9 ", 
                      path_to_data=path_to_data, activation='celu', channels=512, dropout=.0,
                      padding=4, gain=.4, turn_point=500, final_point=3000, minimal_gain=.02, temprature=.125)

study = Study(cifar10_algorithm_generator, args)

2022-06-20 21:04:15.526 | INFO     | src.beam.experiment:__init__:85 - Hyperparameter Optimization
2022-06-20 21:04:15.527 | INFO     | src.beam.experiment:__init__:86 - beam project: cifar10
2022-06-20 21:04:15.529 | INFO     | src.beam.experiment:__init__:87 - Experiment Hyperparameters
2022-06-20 21:04:15.530 | INFO     | src.beam.experiment:__init__:90 - project_name: cifar10
2022-06-20 21:04:15.530 | INFO     | src.beam.experiment:__init__:90 - identifier: debug_hp_optimization_20220620_210415
2022-06-20 21:04:15.531 | INFO     | src.beam.experiment:__init__:90 - algorithm: CIFAR10Algorithm
2022-06-20 21:04:15.532 | INFO     | src.beam.experiment:__init__:90 - mp_port: None
2022-06-20 21:04:15.533 | INFO     | src.beam.experiment:__init__:90 - root_dir: /localdata/elads/data/cifar10
2022-06-20 21:04:15.533 | INFO     | src.beam.experiment:__init__:90 - reload: False
2022-06-20 21:04:15.534 | INFO     | src.beam.experiment:__init__:90 - resume: -1
2022-06-20 21:04:15.535 | INFO    

In [ ]:
# hebo = HEBOSearch(metric="mean_accuracy", mode="max")

# analysis = study.tune(config={"lr_dense": tune.loguniform(1e-3, 2e-2),
#                               "weight_decay": tune.loguniform(1e-6, 1e-4),
#                               "gamma": tune.loguniform(.1, .9),
#                               "dropout": tune.uniform(0, .75),
#                               "scale_down": tune.uniform(0.4, .7),
#                               "scale_up": tune.uniform(0.9, 1.2),
#                               "ratio_down": tune.uniform(0.7, .95),
#                               "ratio_up": tune.uniform(1.05, 1.4),
#                               "channels": tune.choice([128, 256, 512]),
#                               "batch_size": tune.choice([512, 1024, 2048]),},
#                        metric="mean_accuracy",
#                        max_concurrent_trials=4,
#                        resources_per_trial={"gpu": 1},
#                        mode="max",
#                        search_alg=hebo,
#                       progress_reporter=JupyterNotebookReporter(overwrite=True),
#                        num_samples=400)


analysis = study.tune(config={"lr_dense": tune.loguniform(1e-4, 2e-2),
                              "weight_decay": tune.loguniform(1e-6, 1e-2),
                              "beta1": tune.loguniform(.85, .95),
                              "gain": tune.uniform(0.2, .2),
                              "temprature": tune.uniform(0.05, 5.),
                              "minimal_gain": tune.loguniform(.01, .1),
                              "channels": tune.choice([256, 512, 1024]),
                              "padding": tune.choice([4, 6, 8]),
                              "turn_point": tune.choice([256, 512, 1024]),
                              "final_point": tune.choice([2048, 2048*1.5, 2048*2]),
                              "batch_size": tune.choice([512, 1024, 2048]),
                              "activation": tune.choice(['relu', 'celu', 'gelu']),},
                       metric="mean_accuracy",
                       max_concurrent_trials=8,
                       resources_per_trial={"gpu": 1},
                       mode="max",
                       search_alg=OptunaSearch(),
                      progress_reporter=JupyterNotebookReporter(overwrite=True),
                       num_samples=400)


Trial name,status,loc,activation,batch_size,beta1,channels,final_point,gain,lr_dense,minimal_gain,padding,temprature,turn_point,weight_decay,acc,iter,total time (s)
runner_tune_6c7e62ca,RUNNING,172.17.0.2:24720,relu,512,0.850039,1024,2048,0.2,0.00517562,0.0308595,6,1.82263,1024,0.000333747,0.929599,27,326.542
runner_tune_c7b45aa0,RUNNING,172.17.0.2:24775,relu,512,0.852321,1024,2048,0.2,0.00717215,0.0351799,6,1.71532,512,0.000459183,0.907049,21,261.575
runner_tune_cf1b69ba,RUNNING,172.17.0.2:24640,relu,512,0.850407,1024,2048,0.2,0.00499638,0.0301177,6,2.03082,1024,0.000346404,0.928622,39,476.122
runner_tune_ea99898c,RUNNING,172.17.0.2:24832,relu,512,0.850901,1024,2048,0.2,0.0102751,0.037915,6,1.89948,1024,0.000533225,0.844993,15,204.735
runner_tune_0fd686f0,PENDING,,relu,512,0.850103,1024,2048,0.2,0.0102876,0.045788,6,3.40604,1024,1.42559e-05,,,
runner_tune_0e347ab2,TERMINATED,172.17.0.2:24035,gelu,512,0.868141,512,3072,0.2,0.000143155,0.0144721,8,0.553062,512,1.0415e-05,0.886452,40,244.701
runner_tune_12a098fc,TERMINATED,172.17.0.2:23678,gelu,1024,0.948043,512,4096,0.2,0.00237786,0.0175715,6,0.279561,1024,0.00780286,0.917969,40,245.937
runner_tune_1502fc2a,TERMINATED,172.17.0.2:23718,celu,2048,0.873043,1024,4096,0.2,0.000252165,0.0135683,4,4.74368,256,0.00322105,0.891724,40,454.348
runner_tune_155ba23e,TERMINATED,172.17.0.2:24087,relu,512,0.859167,1024,3072,0.2,0.00177057,0.0546031,6,2.41898,1024,1.29446e-06,0.932351,40,488.752
runner_tune_17656412,TERMINATED,172.17.0.2:23804,gelu,2048,0.895943,1024,2048,0.2,0.00604309,0.0103357,6,3.49494,1024,0.00253636,0.867065,40,459.089
